In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal/research'

In [3]:
os.chdir('../.')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataAugConfig:
    root_dir: Path
    train_data: Path
    test_data: Path
    val_data: Path
    train_data_aug: Path
    test_data_aug: Path
    val_data_aug: Path
    horizontal_flip: bool
    vertical_flip: bool
    rotation_range: int
    zoom_range: float
    shear_range: float
    width_shift_range: float
    height_shift_range: float
    target_size: tuple

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

2024-09-18 22:13:49,297 - INFO - start of looging


In [7]:
class ConfigurationManager:
    def __init__(self,confi_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(confi_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config['data_root']])

    
    def get_data_aug(self) -> DataAugConfig:
        config = self.config.data_aug
        params = self.params.AUGMENTATION

        model_config = DataAugConfig(
            root_dir=config.root_dir,
            train_data= config.train_data,
            test_data=config.test_data,
            val_data=config.val_data,
            train_data_aug=config.train_data_aug,
            test_data_aug=config.test_data_aug,
            val_data_aug=config.val_data_aug,
            horizontal_flip=params.horizontal_flip,
            vertical_flip=params.vertical_flip,
            rotation_range=params.rotation_range,
            zoom_range=params.zoom_range,
            shear_range=params.shear_range,
            width_shift_range=params.width_shift_range,
            height_shift_range=params.height_shift_range,
            target_size=params.target_size
        )

        return model_config
        

In [8]:
from log import logger
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np

In [9]:
# from keras.preprocessing.image import ImageDataGenerator
class DataAugmentation:
    def __init__(self,config:DataAugConfig):
        self.config = config

    def augment_data(self):

        for path in [
            self.config.train_data_aug,
            self.config.test_data_aug,
            self.config.val_data_aug
        ]:
            Path(path).mkdir(parents=True, exist_ok=True)

        train_data= self.config.train_data
        test_data=self.config.test_data
        val_data=self.config.val_data
        train_data_aug=self.config.train_data_aug
        test_data_aug=self.config.test_data_aug
        val_data_aug=self.config.val_data_aug
        horizontal_flip=self.config.horizontal_flip
        vertical_flip=self.config.vertical_flip
        rotation_range=self.config.rotation_range
        zoom_range=float(self.config.zoom_range)
        shear_range=self.config.shear_range
        width_shift_range=self.config.width_shift_range
        height_shift_range=self.config.height_shift_range
        target_size=tuple(self.config.target_size)

        train_datagen = ImageDataGenerator(
            rescale=1./255,
            horizontal_flip=horizontal_flip,
            vertical_flip=vertical_flip,
            rotation_range=rotation_range,
            zoom_range=zoom_range,
            shear_range=shear_range,
            width_shift_range=width_shift_range,
            height_shift_range=height_shift_range
        )

        test_datagen = ImageDataGenerator(
            rescale=1./255
        )

        val_datagen = ImageDataGenerator(
            rescale=1./255
        )

        train_data_aug = train_datagen.flow_from_directory(
            train_data,
            target_size=target_size
        )

        test_data_aug = test_datagen.flow_from_directory(
            test_data,
            target_size=target_size
        )

        val_data_aug = val_datagen.flow_from_directory(
            val_data,
            target_size=target_size
        )

        return train_data_aug, test_data_aug, val_data_aug
    
        



In [10]:
try:
    config = ConfigurationManager()
    data_aug_config = config.get_data_aug()
    data_au = DataAugmentation(config=data_aug_config)
    data_au.augment_data()
except Exception as e:
    raise e

2024-09-18 22:13:52,567 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-18 22:13:52,568 - INFO - Yaml file: params.yaml loaded successfully
2024-09-18 22:13:52,569 - INFO - created directory at: data
Found 272 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 58 images belonging to 2 classes.
